# Part 2 of Amini Cocoa Contamination Challenge Solution By SPECIALZ🔥🔥🙌🏽 

## 📈 Inference

### Install Necessary Packages

In [1]:
!pip -q install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-c

In [2]:
!pip -q install ensemble_boxes

### Import Necessary Packages

In [3]:
from collections import defaultdict
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
import sklearn.metrics as skm
import torch
import cv2
import yaml
import albumentations as A
from ultralytics.data.build import YOLODataset
import ultralytics.data.build as build
from ultralytics import YOLO
from ensemble_boxes import *

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 📁  Set configurations and seed

In [4]:
N_SPLITS = 5
RANDOM_STATE = 42
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False

fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 📁 Directory and Data Loading

In [5]:
DIR_DATA = '/kaggle/input/amini-cocoa-contamination-challenge'
df_test = pd.read_csv(f'{DIR_DATA}/Test.csv')
df_test['path'] = f'{DIR_DATA}/dataset/images/test/'+df_test.Image_ID
df_test.shape

(1626, 10)

In [6]:
assert len(df_test) == df_test.Image_ID.nunique()

### ✅ Explanation
This line checks that every row in the df_test DataFrame has a unique Image_ID.

<code>len(df_test)</code> gives the total number of rows.

<code>df_test.Image_ID.nunique()</code> counts the number of unique Image_ID values.

If these two numbers are not equal, the assertion will raise an AssertionError, indicating there are duplicate image entries in the test set — which may lead to data leakage or evaluation issues.

In [7]:
# List of image sizes to infer on
IMGSZS =  [288,384,448,480,544,576,608,640,672,736,768,800,832,864,896]
len(IMGSZS)

15

### 🧠 Explanation
IMGSZS is a list of different image resolutions that will be used for inference.

These varying sizes can help evaluate the model's robustness across different input scales — a technique often used in test-time augmentation or ensembling.

<code>len(IMGSZS)</code> returns the total number of image sizes in the list, which is 15 in this case.



In [8]:
# Define mappings between class names and their corresponding IDs
class2id = {'healthy': 0, 'anthracnose': 1, 'cssvd': 2}
id2class = {v:k for k,v in class2id.items()}
print(id2class)

{0: 'healthy', 1: 'anthracnose', 2: 'cssvd'}


### 🧠 Explanation
<code>class2id</code> is a dictionary that maps class names to numeric IDs.

<code>id2class</code>  is the reverse mapping, converting numeric IDs back to class names. This is useful for interpreting predictions.

## 🔍 Function: run_model_on_test
This function runs a trained YOLO model on a test dataset across multiple image sizes.

### 📥 Inputs:
<code>df_test</code>: A DataFrame containing test image paths and IDs.

<code>model_path</code>: Path to the trained YOLO model weights.

<code>all_data</code>: A dictionary to store detection results for different image sizes.

### ⚙️ Steps:
1. Loads the YOLO model in evaluation mode.

2. Loops through each test image and infers using multiple resolutions defined in IMGSZS.

For each image size:

1. Runs the model with specified parameters (e.g., augment=True, iou=0.5).

2. Extracts bounding boxes, classes, and confidence scores.

3. Applies a confidence threshold (minconf).

4. If no objects are detected, defaults to 'healthy' class with zero bounding box.

5. Otherwise, logs each detection (class, confidence, bounding box coordinates).

6. Appends the results to all_data under the corresponding image size.

### 📤 Output:
Returns all_data, populated with detection results for each image and size.



In [9]:
def run_model_on_test(df_test, model_path, all_data):
    model = YOLO(model_path, task='detect')
    model.eval()
    model.training = False
    minconf = 0.0

    for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
        img = cv2.imread(row.path)
        h, w, _ = img.shape

        for imgsz in IMGSZS:
            results = model(img, imgsz=imgsz, conf=minconf, augment=True, iou=0.5, max_det=1500, verbose=False)[0]
            boxes = results.boxes.xyxy.cpu().numpy()
            classes = results.boxes.cls.cpu().numpy()
            confidences = results.boxes.conf.cpu().numpy()

            ixs = confidences >= minconf
            boxes = boxes[ixs]
            classes = classes[ixs]
            confidences = confidences[ixs]

            if len(boxes) == 0:
                all_data[imgsz].append({
                    'Image_ID': row.Image_ID, 'class': 'healthy', 'confidence': 0,
                    'ymin': 0, 'xmin': 0, 'ymax': 0, 'xmax': 0
                })
            else:
                for box, cls, conf in zip(boxes, classes, confidences):
                    x1, y1, x2, y2 = box
                    detected_class = id2class[int(cls)]
                    all_data[imgsz].append({
                        'Image_ID': row.Image_ID, 'class': detected_class, 'confidence': conf,
                        'ymin': y1, 'xmin': x1, 'ymax': y2, 'xmax': x2
                    })

    return all_data

## 🧪 Inference on Test Set with Multiple Models
This code runs inference on the test set using two trained YOLO models (model1 and model2) and collects predictions for further analysis or ensembling.

#### 🗂 Steps:
<code>defaultdict(list)</code> initializes a dictionary where each value is an empty list by default, useful for storing results grouped by image sizes (IMGSZS).

<code>run_model_on_test(...)</code> runs each model on the test DataFrame df_test, storing predictions like bounding boxes, classes, and confidence scores into the corresponding dictionary (all_data_model1 or all_data_model2).

Each dictionary will contain structured detection outputs per image size, useful for downstream tasks like post-processing, majority voting, or performance comparison between models.

Make sure to take the right models from the model development notebook

I named ours:
```yaml
1. Model with 0.821 CV: '/kaggle/input/amini-cocoa-0.821-model/pytorch/default/1/best.pt'
2. Model with 0.818 CV: '/kaggle/input/yolo11l-weighteddataset-640imgsz-100epochs/pytorch/default/1/yolo11_balanced_best.pt'
```

In [10]:
all_data_model1 = defaultdict(list)
all_data_model2 = defaultdict(list)

all_data_model1 = run_model_on_test(df_test, '/kaggle/input/amini-cocoa-0.821-model/pytorch/default/1/best.pt', all_data_model1)
all_data_model2 = run_model_on_test(df_test, '/kaggle/input/yolo11l-weighteddataset-640imgsz-100epochs/pytorch/default/1/yolo11_balanced_best.pt', all_data_model2)


100%|██████████| 1626/1626 [38:44<00:00,  1.43s/it]


In [11]:
!mkdir -p test_preds
for imgsz in IMGSZS:
    preds1 = pd.DataFrame(all_data_model1[imgsz])
    preds2 = pd.DataFrame(all_data_model2[imgsz])
    preds1.to_csv(f'test_preds/preds_model1_{FOLD}_{imgsz}.csv', index=False)
    preds2.to_csv(f'test_preds/preds_model2_{FOLD}_{imgsz}.csv', index=False)
    print(f"{imgsz}: model1={preds1.shape}, model2={preds2.shape}")

288: model1=(462869, 7), model2=(439757, 7)
384: model1=(562999, 7), model2=(544198, 7)
448: model1=(852248, 7), model2=(851033, 7)
480: model1=(929564, 7), model2=(915605, 7)
544: model1=(977838, 7), model2=(1007606, 7)
576: model1=(1080439, 7), model2=(1090088, 7)
608: model1=(1227250, 7), model2=(1235520, 7)
640: model1=(1124869, 7), model2=(1125167, 7)
672: model1=(1331880, 7), model2=(1338767, 7)
736: model1=(1609241, 7), model2=(1625663, 7)
768: model1=(1357174, 7), model2=(1388915, 7)
800: model1=(1560960, 7), model2=(1590612, 7)
832: model1=(1785374, 7), model2=(1799329, 7)
864: model1=(2002817, 7), model2=(1990378, 7)
896: model1=(1816360, 7), model2=(1852289, 7)


## 📦 Save YOLO Test Predictions for Each Image Size
This snippet above does the following:

Creates a test_preds directory (if it doesn't already exist).

Iterates over all defined image sizes (IMGSZS).

Converts predictions stored in all_data_model1 and all_data_model2 to DataFrames.

Saves each DataFrame as a CSV file named with the model, fold, and image size.

Prints the shape of each prediction file for confirmation.

This step is useful for debugging and later evaluation or ensembling of predictions across different image sizes and models.



## 📦 Function: merge_bboxes
This function fuses bounding box predictions from multiple models using Weighted Boxes Fusion (WBF).

### Parameters:
<code>dfs</code>: List of prediction DataFrames from different models.

<code>iou_thr</code>: IoU threshold for WBF.

<code>skip_box_thr</code>: Confidence threshold to skip low-confidence boxes.

<code>df_meta</code> (optional): Placeholder for additional metadata.

### Steps:
Collects all unique Image_IDs.

For each image, collects and normalizes bounding boxes, scores, and class labels from all models.

Applies WBF to merge overlapping predictions.

Denormalizes and reconstructs final bounding boxes.

Returns a unified DataFrame of predictions for all test images.

This method improves final prediction quality by combining multiple model outputs instead of relying on one.

In [12]:
def merge_bboxes(dfs,iou_thr,skip_box_thr,df_meta=None):
    wb_boxes = []; wb_scores = []; wb_labels = []
    df_res_wbf = []
    image_ids = pd.concat(dfs).Image_ID.unique().tolist()
    weights = None

    cols = ['xmin','ymin','xmax','ymax']

    for Image_ID in tqdm(image_ids):
      boxes_list = []; scores_list = []; labels_list = []
      h,w,_ = cv2.imread(f'{DIR_DATA}/dataset/images/test/{Image_ID}').shape
      for ix_res, df_res in enumerate(dfs):
          d = df_res[df_res.Image_ID==Image_ID].copy()

          boxes = d[cols].values
          scores = d.confidence.tolist()
          labels = d['class'].map(class2id).values.tolist()

          boxes_ = []
          scores_ = []
          labels_ = []
          for i,box in enumerate(boxes):
            if box[2]> box[0] and box[3]>box[1]:
              boxes_.append(box)
              scores_.append(scores[i])
              labels_.append(labels[i])

          boxes = boxes_
          scores = scores_
          labels = labels_

          boxes = [[x[0]/w,x[1]/h,x[2]/w,x[3]/h] for x in boxes]

          boxes_list.append(boxes)
          scores_list.append(scores)
          labels_list.append(labels)

      if len(boxes_list)>0:
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=[1 for i in range(len(boxes_list))], iou_thr=iou_thr, skip_box_thr=skip_box_thr)

      else:
        boxes = []

      boxes = [[x[0]*w,x[1]*h,x[2]*w,x[3]*h] for x in boxes]
      ws = [b[2]-b[0] for b in boxes]
      hs = [b[3]-b[1] for b in boxes]

      if len(boxes)==0:
        d_res = pd.DataFrame([{ 'Image_ID': Image_ID, 'class': 'healthy', 'confidence': 0.5, 'ymin': 0, 'xmin': 0, 'ymax': 0, 'xmax': 0 }])
      else:
        xmin = np.array(boxes)[:,0]
        ymin = np.array(boxes)[:,1]
        xmax = np.array(boxes)[:,2]
        ymax = np.array(boxes)[:,3]

        d_res = pd.DataFrame(dict(Image_ID=Image_ID,confidence=scores,ymin=ymin,xmin=xmin,ymax=ymax,xmax=xmax))
        d_res['class'] =  [id2class[l] for l in labels ]

      df_res_wbf.append(d_res)


    df_res_wbf = pd.concat(df_res_wbf)
    return df_res_wbf

## 🧪 Ensembling Predictions with WBF
This block:

Loads prediction CSVs for different image sizes from two YOLO models.

Aggregates them into a single list of DataFrames.

Merges the predictions using Weighted Boxes Fusion to enhance final detection accuracy.

Outputs the final ensembled predictions into <code>submission.csv</code> for evaluation or submission.

This method leverages diversity across scales and models to produce more robust results.

In [13]:
dfs = []
for imgsz in IMGSZS:
    df1 = pd.read_csv(f'test_preds/preds_model1_{FOLD}_{imgsz}.csv')
    df2 = pd.read_csv(f'test_preds/preds_model2_{FOLD}_{imgsz}.csv')
    dfs.extend([df1, df2])

iou_thr = 0.558
skip_box_thr = 0.005

df_res_wbf = merge_bboxes(dfs, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
df_res_wbf.to_csv('submission.csv', index=False)
print("Saved to submission.csv")

100%|██████████| 1626/1626 [1:12:53<00:00,  2.69s/it]


Saved to submission.csv


### And Don't Just Have Good Day, Have A Great Day!
## Thank You!